<a href="https://colab.research.google.com/github/fagonzalezo/qmc/blob/master/examples/QM-DM-ClassifierEig-test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install qmc if running in Google Colab

try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
    !pip install git+https://github.com/fagonzalezo/qmc.git
else:
    import sys
    sys.path.insert(0, "../")

In [2]:
from sklearn.metrics import accuracy_score
from sklearn.datasets import make_blobs, make_moons, make_circles
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_digits
import numpy as np
import tensorflow as tf
import qmc.tf.layers as layers
import qmc.tf.models as models


# Digits

In [3]:
X,y = load_digits(return_X_y=True)
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.33, stratify=y, random_state=42)

print(X_train.shape)
y_train = y_train[:, np.newaxis]
y_test = y_test[:, np.newaxis]
y_train_bin = tf.reshape (tf.keras.backend.one_hot(y_train, 10), (-1,10))
y_test_bin = tf.reshape (tf.keras.backend.one_hot(y_test, 10), (-1,10))
print(y_train.shape)
print(y_train_bin.shape)

(1203, 64)
(1203, 1)
(1203, 10)


In [4]:
fm_x = layers.QFeatureMapRFF(64, dim=150, gamma=0.375, random_state=17)
fm_y = layers.QFeatureMapOneHot(num_classes=10)
qmc_dig = models.QMClassifier(fm_x=fm_x, fm_y=fm_y, dim_x=150, dim_y=10)
qmc_dig.compile()
qmc_dig.fit(X_train, y_train, epochs=1)
out = qmc_dig.predict(X_test)
print(accuracy_score(y_test, np.argmax(out, axis=1)))

38/38 [==============================] - 3s 77ms/step
0.9377104377104377


In [5]:
qmc_sgd_dig = models.QMClassifierSGD(input_dim=64, dim_x=150, dim_y=10,
                                     num_eig=10, gamma=0.375, random_state=17)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
qmc_sgd_dig.compile(optimizer, loss=tf.keras.losses.CategoricalCrossentropy())
qmc_sgd_dig.fit(X_train, y_train_bin, epochs=10)
out = qmc_sgd_dig.predict(X_test)
print(accuracy_score(y_test, np.argmax(out, axis=1)))

Epoch 1/10
38/38 [==============================] - 0s 1ms/step - loss: 1.4909
Epoch 2/10
38/38 [==============================] - 0s 1ms/step - loss: 0.4057
Epoch 3/10
38/38 [==============================] - 0s 1ms/step - loss: 0.2784
Epoch 4/10
38/38 [==============================] - 0s 1ms/step - loss: 0.2234
Epoch 5/10
38/38 [==============================] - 0s 1ms/step - loss: 0.1894
Epoch 6/10
38/38 [==============================] - 0s 1ms/step - loss: 0.1553
Epoch 7/10
38/38 [==============================] - 0s 1ms/step - loss: 0.1302
Epoch 8/10
38/38 [==============================] - 0s 1ms/step - loss: 0.1178
Epoch 9/10
38/38 [==============================] - 0s 1ms/step - loss: 0.1096
Epoch 10/10
38/38 [==============================] - 0s 1ms/step - loss: 0.0938
0.9797979797979798


In [9]:
inputs = tf.keras.Input(shape=(64,))
fm_x = layers.QFeatureMapRFF(64, dim=150, gamma=0.375, random_state=17)
psi_x = fm_x(inputs)
ones = tf.fill((tf.shape(inputs)[0], 1), 1.0)
rho_x = tf.keras.layers.concatenate((ones, psi_x), axis=1)
rho_x = tf.expand_dims(rho_x, axis=-1)
qmdmc = layers.QMeasureDMClassifEig(dim_x=150, dim_y=10, eig_out=10, num_eig=10)
rho_y = qmdmc(rho_x)
y_w = rho_y[:, 0, :] # shape (b, d_in)
y_v = rho_y[:, 1:, :] # shape (b, dim_x, d_in)
probs = tf.einsum('...j,...ij,...ij->...i', y_w, y_v, tf.math.conj(y_v))
qmdmc = tf.keras.Model(inputs=inputs, outputs=probs)

In [10]:
qmdmc.compile(optimizer, loss=tf.keras.losses.CategoricalCrossentropy())
qmdmc.fit(X_train, y_train_bin, epochs=10)
out = qmdmc.predict(X_test)
print(accuracy_score(y_test, np.argmax(out, axis=1)))

Epoch 1/10
38/38 [==============================] - 1s 2ms/step - loss: 1.1872
Epoch 2/10
38/38 [==============================] - 0s 2ms/step - loss: 0.3227
Epoch 3/10
38/38 [==============================] - 0s 2ms/step - loss: 0.1833
Epoch 4/10
38/38 [==============================] - 0s 2ms/step - loss: 0.1331
Epoch 5/10
38/38 [==============================] - 0s 2ms/step - loss: 0.1142
Epoch 6/10
38/38 [==============================] - 0s 2ms/step - loss: 0.0910
Epoch 7/10
38/38 [==============================] - 0s 2ms/step - loss: 0.0823
Epoch 8/10
38/38 [==============================] - 0s 2ms/step - loss: 0.0683
Epoch 9/10
38/38 [==============================] - 0s 2ms/step - loss: 0.0633
Epoch 10/10
38/38 [==============================] - 0s 2ms/step - loss: 0.0594
0.9747474747474747


# MNIST

In [15]:
from tensorflow.keras.datasets import mnist
from sklearn.preprocessing import OneHotEncoder

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape((60000,784))
X_test = X_test.reshape((10000,784))


X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

print("shape X_train : ", X_train.shape)
print("shape y_train : ", y_train.shape)
print("shape X_test : ", X_test.shape)
print("shape y_test : ", y_test.shape)
transformer = OneHotEncoder(sparse=False)
y_train_bin = transformer.fit_transform(y_train[:, np.newaxis])

shape X_train :  (60000, 784)
shape y_train :  (60000,)
shape X_test :  (10000, 784)
shape y_test :  (10000,)


In [25]:
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [13]:
def create_model(input_dim, num_classes, component_dim=100, gamma=1, lr=0.01, decay=0.,
                  random_state=None, eig_percentage=0, initialize_with_rff=False,
                  type_of_rff="rff", fix_rff=False):
    '''This is a model generating function so that we can search over neural net
    parameters and architecture'''

    num_eig = round(eig_percentage * component_dim)

    opt = tf.keras.optimizers.Adam(lr=lr, beta_1=0.9, beta_2=0.999, decay=decay)

    if type_of_rff == 'rff':
        fm_x = layers.QFeatureMapRFF(input_dim, dim=component_dim, gamma=gamma, random_state=random_state)
    #else:
    #    fm_x = layers.QFeatureMapORF(input_dim, dim=component_dim, gamma=gamma, random_state=random_state)

    if initialize_with_rff:
        qmkdc = models.DMKDClassifier(fm_x=fm_x, dim_x=component_dim, num_classes=num_classes)
        qmkdc.compile()
        qmkdc.fit(X_train, y_train, epochs=1, batch_size=batch_size)

    qmkdc1 = models.DMKDClassifierSGD(input_dim=input_dim, dim_x=component_dim, num_eig=num_eig,
                                      num_classes=num_classes, gamma=gamma, 
                                      random_state=random_state)

    if fix_rff:
        qmkdc1.layers[0].trainable = False

    qmkdc1.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=opt, metrics=['accuracy'])

    if initialize_with_rff:
        qmkdc1.set_rhos(qmkdc.get_rhos())

    # qmkdc1.fit(X_train, y_train_bin, epochs=epochs, batch_size=batch_size)

    return qmkdc1

In [26]:
input_dim = 784
component_dim = 784
num_classes = 10
gamma = 2**-5
random_state=0
num_eig=0
batch_size=32
epochs = 10
model_mnist = create_model(input_dim, num_classes, component_dim=round(input_dim/2), 
                           gamma=2**-9, lr=0.00001, decay=1.000000e-06,
                           random_state=None, eig_percentage=0.1, 
                           initialize_with_rff=False,
                           type_of_rff="rff", fix_rff=False)

/Users/fgonza/miniforge3/envs/tf_x86/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:381: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [27]:
opt = tf.keras.optimizers.RMSprop(lr=0.00000001,  decay=1.000000e-06)
model_mnist.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=opt, metrics=['accuracy'])

In [28]:
history = model_mnist.fit(X_train, y_train_bin, epochs=10, batch_size=32)

Epoch 1/10
1875/1875 [==============================] - 8s 3ms/step - loss: 2.3848 - accuracy: 0.0869
Epoch 2/10
1875/1875 [==============================] - 6s 3ms/step - loss: 2.3807 - accuracy: 0.0874
Epoch 3/10
1875/1875 [==============================] - 6s 3ms/step - loss: 2.3759 - accuracy: 0.0857
Epoch 4/10
1875/1875 [==============================] - 7s 4ms/step - loss: 2.3722 - accuracy: 0.0880
Epoch 5/10
1875/1875 [==============================] - 6s 3ms/step - loss: 2.3715 - accuracy: 0.0873
Epoch 6/10
1875/1875 [==============================] - 6s 3ms/step - loss: 2.3640 - accuracy: 0.0874
Epoch 7/10
1875/1875 [==============================] - 6s 3ms/step - loss: 2.3619 - accuracy: 0.0879
Epoch 8/10
1875/1875 [==============================] - 6s 3ms/step - loss: 2.3569 - accuracy: 0.0869
Epoch 9/10
1875/1875 [==============================] - 6s 3ms/step - loss: 2.3525 - accuracy: 0.0887
Epoch 10/10
1875/1875 [==============================] - 6s 3ms/step - loss: 2.350

In [29]:
inputs = tf.keras.Input(shape=(784,))
fm_x = layers.QFeatureMapRFF(784, dim=784, gamma=2**-5, random_state=17)
psi_x = fm_x(inputs)
ones = tf.fill((tf.shape(inputs)[0], 1), 1.0)
rho_x = tf.keras.layers.concatenate((ones, psi_x), axis=1)
rho_x = tf.expand_dims(rho_x, axis=-1)
qmdmc = layers.QMeasureDMClassifEig(dim_x=784, dim_y=10, eig_out=80, num_eig=80)
rho_y = qmdmc(rho_x)
y_w = rho_y[:, 0, :] # shape (b, d_in)
y_v = rho_y[:, 1:, :] # shape (b, dim_x, d_in)
probs = tf.einsum('...j,...ij,...ij->...i', y_w, y_v, tf.math.conj(y_v))
qmdmc = tf.keras.Model(inputs=inputs, outputs=probs)

In [30]:
qmdmc.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=opt, metrics=['accuracy'])
history = qmdmc.fit(X_train, y_train_bin, epochs=10, batch_size=32)

Epoch 1/10
1875/1875 [==============================] - 61s 32ms/step - loss: 2.3367 - accuracy: 0.0923
Epoch 2/10
1875/1875 [==============================] - 61s 32ms/step - loss: 2.3368 - accuracy: 0.0931
Epoch 3/10
1875/1875 [==============================] - 60s 32ms/step - loss: 2.3362 - accuracy: 0.0931
Epoch 4/10
1875/1875 [==============================] - 60s 32ms/step - loss: 2.3368 - accuracy: 0.0942
Epoch 5/10
1875/1875 [==============================] - 61s 32ms/step - loss: 2.3364 - accuracy: 0.0914
Epoch 6/10
1875/1875 [==============================] - 61s 32ms/step - loss: 2.3335 - accuracy: 0.0953
Epoch 7/10
1875/1875 [==============================] - 59s 32ms/step - loss: 2.3343 - accuracy: 0.0925
Epoch 8/10
1875/1875 [==============================] - 60s 32ms/step - loss: 2.3345 - accuracy: 0.0957
Epoch 9/10
1875/1875 [==============================] - 60s 32ms/step - loss: 2.3338 - accuracy: 0.0956
Epoch 10/10
1875/1875 [==============================] - 62s 33m